In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [14.6 kB]
Get:12 http://security.ubuntu.

In [ ]:
# Start Spark session
# download a Postgres driver that will allow Spark to interact with Postgres:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

# Spark session with an additional option that adds the driver to Spark:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Big_Data").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

--2020-10-01 16:32:45--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.33MB/s    in 0.2s    

2020-10-01 16:32:45 (5.33 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz'

In [ ]:
spark.sparkContext.addFile(url)
sport_df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Sports_v1_00.tsv.gz'), sep='\t', header=True)
sport_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [ ]:
clean_sport_df = sport_df.dropna().dropDuplicates()
clean_sport_df.count()

4849000

review ID table has:

> review_id TEXT PRIMARY KEY NOT NULL,  
customer_id INTEGER,  
product_id TEXT,  
product_parent INTEGER,  
review_date DATE -- this should be in the formate yyyy-mm-dd

product table has:
>   product_id TEXT PRIMARY KEY NOT NULL UNIQUE,  product_title TEXT

customer table has:
>   customer_id INT PRIMARY KEY NOT NULL UNIQUE,  customer_count INT

vine table has:
>   review_id TEXT PRIMARY KEY,  
star_rating INTEGER,  
helpful_votes INTEGER,  
total_votes INTEGER,  
vine TEXT





In [ ]:
# Import types and col function
from pyspark.sql.types import StringType, IntegerType, DateType
from pyspark.sql.functions import col

In [ ]:
# create review table
review_df = clean_sport_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_df = review_df.dropna().dropDuplicates()
review_df = review_df.withColumn("review_id",col("review_id").cast(StringType()))\
  .withColumn("customer_id",col("customer_id").cast(IntegerType()))\
  .withColumn("product_id",col("product_id").cast(StringType()))\
  .withColumn("product_parent",col("product_parent").cast(IntegerType()))\
  .withColumn("review_date",col("review_date").cast(DateType()))
review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R11AOFZKETHHYN|   51608413|B007DM6S72|      59327699| 2015-06-07|
|R11T8C4WBP7K9P|   12178538|B001391PRO|      75255285| 2014-10-22|
|R11V8I66U7X7DQ|   25141989|B00HA2ZKCO|     997278452| 2014-08-24|
|R12EGTQFGVSYYV|   19237355|B00JSRHXNS|     883435825| 2014-11-30|
|R131ZLTDCB8V40|    4257729|B00012PDMW|     272857416| 2014-03-21|
|R16VPH7EFHZHTJ|   21704149|B003TX0V5O|     852045139| 2013-01-04|
|R16WDO1W94KM7R|   38518912|B001B693V4|     284586274| 2014-11-17|
|R1A2WGX6RZ7ZS7|   18284374|B00066X6EA|     896007353| 2014-05-06|
|R1BHVVU2CQOODD|    7201753|B00FAI53G8|     874770006| 2015-07-09|
|R1BXKR9FJLBKV5|   14396754|B003TVT1KC|     500138151| 2012-09-06|
|R1CQ3QQD68XQRF|   12920846|B001NOVF92|     389615357| 2014-09-14|
|R1DZP3ASKMZK2D|   12620577|B00I321M68|     360817361| 2014-12

In [ ]:
# create poduct table
product_df = clean_sport_df.select(['product_id', 'product_title'])
cleaned_product_df = product_df.dropDuplicates(["product_id"])
# Set columns types
cleaned_product_df = cleaned_product_df.withColumn("product_id",col("product_id").cast(StringType()))\
  .withColumn("product_title",col("product_title").cast(StringType()))
cleaned_product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1570340439|Compass & Map Nav...|
|9879000633|Swordmaster - Uni...|
|B00004RAN1|24x7x7 Pro Cage Trap|
|B00005OU7P|Putt and Return P...|
|B00005V3EN|Airzone Trampolin...|
|B00008Z7M2|Koolatron 401619 ...|
|B0000AAEMO|Seahawks Franklin...|
|B0000AI0K1|Outdoor Cap Heavy...|
|B0000ANGL1|NFL Men's New Yor...|
|B0000ANHRR|Heatgear Loose Lo...|
|B0000AU3AK|Everlast Genuine ...|
|B0000AUTWD|Cubby 5011 Mini Mite|
|B0000AVZBW|G- Lox  Deerskin ...|
|B0000AXJT5|Pro Mex Professio...|
|B0000AXNWO|   Ande Fluorocarbon|
|B0000AXUET|Star brite Brush ...|
|B0000AY199|DU-BRO Fishing Tr...|
|B0000AYEE8|Attwood Stainiles...|
|B0000AYGZ1|Attwood Stainless...|
|B0000AZ8LS|Tacstar 4-Shot Re...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# create customer table
from pyspark.sql.functions import col
customer_df = clean_sport_df.groupBy('customer_id').count()
customer_df.sort(col('count')).show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    6060548|    1|
|   16872545|    1|
|   39872740|    1|
|   16254686|    1|
|   11040445|    1|
|   26655197|    1|
|   52960291|    1|
|   16303700|    1|
|   27293091|    1|
|    1171167|    1|
|   19946204|    1|
|   29412283|    1|
|   34593885|    1|
|   36923466|    1|
|   28234212|    1|
|   11395818|    1|
|     498279|    1|
|   15087244|    1|
|   38824581|    1|
|   15313678|    1|
+-----------+-----+
only showing top 20 rows



In [ ]:
# change 'count' column name to 'customer_count' and sort
customer_df = customer_df.withColumnRenamed('count', 'customer_count')
customer_df = customer_df.dropDuplicates(["customer_id"])
# Set columns types
customer_df = customer_df.withColumn("customer_id",col("customer_id").cast(IntegerType()))\
  .withColumn("customer_count",col("customer_count").cast(IntegerType()))
customer_df = customer_df.sort(col('count'), ascending = False)
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   50820654|           552|
|   37651511|           294|
|   26955164|           272|
|   50227539|           257|
|   52870270|           245|
|   25355517|           225|
|   13355404|           193|
|   46160224|           177|
|   26772822|           172|
|   52770861|           170|
|   49335121|           162|
|    1871743|           154|
|   25436988|           144|
|   20032310|           143|
|   19512365|           142|
|   51354368|           140|
|   18453567|           138|
|   29505740|           136|
|   36579538|           136|
|   51591392|           132|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# prepare to validate vine reviews by filtering for only verified purchases 
# to help filter out fake reviews
vine_verified_df = sport_df.filter(sport_df['verified_purchase'] == 'Y')
vine_verified_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|            1|          1|   N|                Y|       Shipped fast.|These are the bes...| 2015-08-31|
|         US|   45813853| RR8V7WR27NXJ5|B008VS8M58|     962442336|Baoer 223 5.56x45...|          Sports|          1|    

In [ ]:
# create vine table for DB
vine_verified_df = sport_df.filter(sport_df['verified_purchase'] == 'Y')
clean_vine_df = vine_verified_df.dropna().dropDuplicates()
vine_filter_df = clean_vine_df.filter(clean_vine_df['vine'] == 'Y')
vine_df = vine_filter_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_final_df = vine_df.withColumn("review_id",col("review_id").cast(StringType()))\
  .withColumn("star_rating",col("star_rating").cast(IntegerType()))\
  .withColumn("helpful_votes",col("helpful_votes").cast(IntegerType()))\
  .withColumn("total_votes",col("total_votes").cast(IntegerType()))\
  .withColumn("vine",col("vine").cast(StringType()))
vine_final_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1R2FIKQC9XIKO|          4|            0|          1|   Y|
|R2AK4UF93O4EGW|          5|            0|          1|   Y|
|R2YCOOKA91SPQ4|          5|            1|          1|   Y|
|R1ITLW7VCY8ZNV|          5|            2|          2|   Y|
| RUVM52QIHJ57R|          5|            1|          1|   Y|
|R2RVYNK5RNM4EX|          5|            0|          0|   Y|
|R1H8QU9G9HLBWK|          5|           43|         49|   Y|
|R2LO3B4TW53KY3|          5|            3|          3|   Y|
|R3FYZFOAA7EZU9|          5|            1|          1|   Y|
|R20N8XGNA7X5EB|          5|            0|          0|   Y|
|R2FCLG4I9PIQSO|          4|            4|          4|   Y|
+--------------+-----------+-------------+-----------+----+



In [ ]:
#  use pySpark to load data into AWS S3

# aws.czi9q1pcbluc.us-east-2.rds.amazonaws.com
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://aws.czi9q1pcbluc.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "postpost", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to reviews table in RDS
review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to products table in RDS
cleaned_product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customers table in RDS
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to Vine reviews table in RDS
vine_final_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Evaluation of the Vine reviews looked at the total number of reviews, average rating, number of votes and length of reviews.  The Vine reviews were removed from the total review population and examined separately.

The length of reviews is the strongest indicator that a review is legitimate and trustworthy.  Second to this is the number of helpful votes.

Since this is such small result of Vine reviews for this category, further analysis isn't justified in this case, trying to see the distribution or look for outliers is not informative with a population size smaller than typical sample sizes.

Of all of the sports product reviews in this dataset, there are only 11 vine reviews, three of which have word counts above the mean.  Spoofed or SPAM reviews are usually shorter in length.  Some websites use rules of thumb like at least 20 words for a review to be accepted.  All of these reviews exceeded that threshold by an order of magnitude.  Furthermore, the shortest review was 265 words.  A typed, full-length page has approximately 250 words.  This is quite lengthy for a review and most any that are this long or longer are likely to be trustworthy reviews.

You can say these sports product fans aren't just fans of the product; they're fanatical about their reviews as well.
